# Learning to Rank on MQ2008 Dataset

In [135]:
# Importing libraries

import pandas as pd
import numpy as np
import itertools
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

## Building train, validation, test

In [60]:
# Extracting (q, d, c) from dataset

def load_data(file_path):
    '''returns a dictionary with keys equal to queries and values equal to features of each doc
    dict of q: (d1, c1), (d2, c2), ..., (dn, cn) where c is relevance and d is doc feature vector'''
    query_feedback = {}
    with open(file_path) as fp:
        line = fp.readline()
        while line:
            parts = line.strip().split(" ")
            c = int(parts[0])
            q = parts[1].split(":")[1]
            s = parts[2:48]
            s = np.array([float(f.split(":")[1]) for f in s])
            if q in query_feedback:
                query_feedback[q].append((s, c))
            else:
                query_feedback[q] = [(s, c)]
            line = fp.readline()
    return query_feedback       

In [176]:
# Building features and targets

def build_pairwise_data(query_dict):
    y = []
    X = []
    for query in query_dict:
        docs = query_dict[query]
        for (d1, c1), (d2, c2) in itertools.combinations(docs, 2):
            X.append(d1 - d2)
            y.append(np.sign(c1 - c2))
    y = np.array(y)
    X = np.array(X)
    return X, y    

In [177]:
data_train = load_data('train.txt')
data_test = load_data('test.txt')
data_val = load_data('vali.txt')

In [178]:
print("Num training queries: ", len(data_train))
print("Num validation queries: ", len(data_val))
print("Num test queries: ", len(data_test))

Num training queries:  471
Num validation queries:  157
Num test queries:  156


In [179]:
X_train, y_train = build_pairwise_data(data_train)
X_test, y_test = build_pairwise_data(data_test)
X_val, y_val = build_pairwise_data(data_val)

In [180]:
X_train.shape, X_test.shape, X_val.shape

((228021, 46), (57434, 46), (45270, 46))

In [181]:
len(y_train == 1)

228021

In [182]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [183]:
np.unique(y_train)

array([-1,  0,  1])

## Training SVM Model and Hyper-parameter Tuning

In [162]:
svm_model = svm.LinearSVC(random_state=0)

In [163]:
svm_model.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

In [164]:
y_pred = svm_model.predict(X_test)
print("Test accuracy = ", accuracy_score(y_pred, y_test))

Test accuracy =  0.7499564717762998


In [165]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [171]:
# Hyper-parameter tuning with validation data

params = {'C': [1, 0.1, 0.01, 0.001, 0.0001]}

svm_model = svm.LinearSVC(random_state=0)
svm_model = GridSearchCV(estimator = svm_model, param_grid = params, cv = 5)
svm_model.fit(X_val, y_val)

print("Best params = ", svm_model.best_params_)

Best params =  {'C': 0.0001}


In [172]:
y_pred = svm_model.predict(X_test)
print("Test accuracy = ", accuracy_score(y_pred, y_test))

Test accuracy =  0.7499564717762998


In [157]:
print("Classification Report:\n", classification_report(y_pred, y_test))

Classification Report:
               precision    recall  f1-score   support

          -1       0.99      0.87      0.93     56823
           1       0.04      0.59      0.08       611

    accuracy                           0.86     57434
   macro avg       0.52      0.73      0.50     57434
weighted avg       0.98      0.86      0.92     57434



In [156]:
print("Confusion Matrix:\n", confusion_matrix(y_pred, y_test))

Confusion Matrix:
 [[49168  7655]
 [  251   360]]


## Model Performance with NDCG@5

In [ ]:
# https://en.wikipedia.org/wiki/Discounted_cumulative_gain